In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp
import random
import math

from numpy import mean, std
from scipy import stats
import scipy as sp

from matplotlib import pyplot
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV,  cross_val_score, RepeatedStratifiedKFold, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import get_scorer, make_scorer, confusion_matrix, classification_report, recall_score, precision_score, accuracy_score, fbeta_score, roc_curve, roc_auc_score, f1_score, confusion_matrix, mean_squared_error, log_loss
from sklearn import svm, datasets
from sklearn.naive_bayes import BernoulliNB
from sklearn.datasets import make_classification
from sklearn.exceptions import ConvergenceWarning
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.exceptions import ConvergenceWarning
from sklearn.naive_bayes import BernoulliNB

from warnings import simplefilter
import imblearn
from imblearn.over_sampling import SMOTE

from sklearn.base import clone
from sklearn.preprocessing import FunctionTransformer



simplefilter("ignore", category = ConvergenceWarning)

In [2]:
class KidronClassifier:
    
    def __init__(self, min_rows = None, min_columns = None, max_estimators = None, estimators = None, base_estimator = None, random_state = None):

        self.min_rows = min_rows
        self.min_columns = min_columns
        self.max_estimators = max_estimators
        self.estimators = estimators
        self.base_estimator = base_estimator 
        self.random_state = random_state 
        #self.fitted_estimators = None
        #self.fitted_columns = None        
        return
             
    def get_params(self, deep = True):
        return {"min_rows": self.min_rows, 
                "min_columns": self.min_columns,
                "max_estimators": self.max_estimators,
                "estimators": self.estimators,
                "base_estimator": self.base_estimator,
                "random_state": self.random_state}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    #build training data for a single model
    def even_data(self, dict_X, length, width, seed):     
        
        labels = dict_X.keys()          
        X = pd.concat([dict_X[label].sample(n = length, random_state = seed) for label in labels]) #length
        
        random.seed(seed)
        columns = random.sample(range(len(X.columns)), width)
        X = X.iloc[:, columns]
        
        y = []
        for label in labels:
            y.extend([label] * length)
        
        return X, y, columns

    #fit 
    def fit(self, X, y):

        #label encoding
        self.le = LabelEncoder()
        self.le.fit(y)
        
        #X = pd.DataFrame(X)
        
        length = X.shape[0] #len(X);
        width  = X.shape[1] #len(X.columns)
        col = min([int(width  ** 0.5) + 1, width])
        row = min([int(length ** 0.5) + 1, length])            
        estimators = row * col
        
        if(self.max_estimators):
            estimators = min([estimators, self.max_estimators])
            
        if(self.estimators is None):
            if(self.base_estimator):
                self.estimators = [clone(self.base_estimator) for i in range(estimators)]
            else:
                self.estimators = [LogisticRegression() for i in range(estimators)]  #default
                                
        dict_X = {}
        for label in self.le.classes_:
            dict_X[label] = X[y == label]
            
        rows = min([len(dict_X[label]) for label in dict_X.keys()])  #maximal length available  
        rows = min(row, length)
        if(self.min_rows):
            rows = max([self.min_rows, rows])
            
        cols = col
        if(self.min_columns):
            cols = max([self.min_columns, cols])
        
        seed, self.fitted_estimators, self.fitted_columns = 0, [], []
        #print(len(self.estimators))
        
        for estimator in self.estimators:

            #get a balanced data
            X_train, y_train, fitted_columns = self.even_data(dict_X, rows, cols, seed)
            
            #fit
            fitted_estimator = estimator.fit(X_train, y_train)
            
            #save fitted
            self.fitted_estimators.append(fitted_estimator)
            self.fitted_columns.append(fitted_columns)
            
            seed += 1
        return
    
    #predict_proba
    def predict_proba(self, X):

        length = len(X)
        labels = len(self.le.classes_)
        
        pred = []
        
        # Predict 'soft' voting with probabilities
        predict_proba = []
        for fitted_estimator, fitted_columns in zip(self.fitted_estimators, self.fitted_columns):             
            fitted_X = X.iloc[:, fitted_columns]
            predict_proba.append(np.asarray(fitted_estimator.predict_proba(fitted_X)).reshape(length * labels, 1))
            
        predict_proba = np.concatenate(predict_proba, axis = 1)
        predict_proba = np.average(predict_proba, axis = 1).reshape(length, labels)
            
        # Convert integer predictions to original labels:
        return predict_proba
    
    #predict
    def predict(self, X):
            
        proba = self.predict_proba(X)
        
        # Convert integer predictions to original labels:
        return self.le.inverse_transform(np.argmax(proba, axis = 1))
    
    #predict, old slow version
    def predict_slow(self, X):

        pred = []
        for i in range(len(X)):

            # Predict 'soft' voting with probabilities
            x = X[i : i + 1]
            predict_proba = np.asarray([estimator.predict_proba(x) for estimator in self.fitted_estimators])
            predict_proba = np.average(predict_proba, axis = 0)
        
            #the indice of the average
            pred.append(np.argmax(predict_proba, axis = 1)[0])        

        # Convert integer predictions to original labels:
        return self.le.inverse_transform(pred)

In [3]:
path = "/Users/yaeerk/Documents/NAYA/classification/"
jobs = 8

In [4]:
df = pd.read_csv(path + 'PROTECT_and_RISK_shared_DEGs_NOT NORM.csv', index_col = 0)

In [1]:
X = df.drop('Diagnosis', axis = 1)
y = df.Diagnosis
y = LabelEncoder().fit_transform(y)
numerical_cols = X.columns.to_list()

NameError: name 'df' is not defined

In [6]:
tab = "  "
testsizes = [0.08, 0.1, 0.15, 0.2]
#randomstates = [132, 400, 1440, 1600, 2500, 3333, 4567]
randomstates = [250, 650, 850, 1050, 1250, 1850, 2050]
features = 50

In [7]:
def dominant_features(X_train, X_test, y_train, t, n):
    genes = {}
    for r in randomstates:
        #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = t, random_state = r)

        for depth in range(2, 6, 2):
            dtc = DecisionTreeClassifier(max_depth = depth).fit(X_train, y_train)
            sort_arr = np.argsort(dtc.feature_importances_)

            for i in range(-3, -1, 1):
                gene  = X.columns[sort_arr[i]]
                value = dtc.feature_importances_[sort_arr[i]]
                if(value <= 0):
                    continue

                if(gene in genes):
                    genes[gene] += value
                else:
                    genes[gene] = value

    features = [(k, v) for k, v in sorted(genes.items(), reverse = True, key = lambda item: item[1])]
    feature_names = [k for k, v in features]
    feature_names = feature_names[:n]
    
    return X_train.drop(X_train.columns.difference(feature_names), 1), X_test.drop(X_test.columns.difference(feature_names), 1)

In [8]:
#drop all features, except to 50
#feature_names = [k for k, v in features]
#feature_names = feature_names[:50]
#X.drop(X.columns.difference(feature_names), 1, inplace = True)
#X.shape

In [9]:
#bagging parameters
n_estimators = [2, 10, 20, 30]
max_samples = [0.6, 0.8, 1.0]
bootstrap = [True, False]
bootstrap_features = [True, False]

#cross validation
cv = 5

#scores we want
scores = ['f1', 'accuracy']

results = pd.DataFrame(columns = ["score", "test score", "train score", "test variance", "train variance", "test rmse", "train rmse", "test log_loss", "train log_loss", "test size", "random state", "estimator", "estimator params"])
results.head()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params


In [10]:
#random forest
max_depths = [10, 20, 30]
min_samples_leafs = [3, 10]

for testsize in testsizes:
  print(f"test size: {testsize}")


  for randomstate in randomstates:
        print(tab * 2 + f"random state: {randomstate}")
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state = randomstate)
        X_train, X_test = dominant_features(X_train, X_test, y_train, testsize, features)

        smote = SMOTE(random_state = randomstate)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        robustscaler = RobustScaler(quantile_range = (1, 99))
        robustscaler.fit(X_train)
                                    
        X_train = robustscaler.transform(X_train)
        X_test  = robustscaler.transform(X_test)

        best_avg_scores = {score : [None] for score in scores}
        
        # Run Grid search for each classifier
        for max_depth in max_depths:
            for min_samples_leaf in min_samples_leafs:
                    
                rf = RandomForestClassifier(max_depth = max_depth, min_samples_leaf = min_samples_leaf, random_state = randomstate)
                cv_results = cross_validate(rf, X_train, y_train, cv = cv, scoring = scores, return_train_score = True, n_jobs = jobs)
 
                for score in scores:
                    avg_score_test = np.mean(cv_results['test_' + score])
                    var_score_test = np.var(cv_results['test_' + score])
                    avg_score_train = np.mean(cv_results['train_' + score])
                    var_score_train = np.var(cv_results['train_' + score])

                    if(best_avg_scores[score][0] is None or avg_score_test > best_avg_scores[score][0]):
                        best_avg_scores[score] = [avg_score_test, var_score_test, avg_score_train, var_score_train, max_depth, min_samples_leaf]

        for score in scores: 
            
            print(tab * 3 + str(score))
            print(tab * 4 + f"CV score: {best_avg_scores[score][0]} using:" + ','.join([str(p) for p in best_avg_scores[score][4:]]))
            print(tab * 5 + f"train score: {best_avg_scores[score][2]} with variance: {best_avg_scores[score][3]}")
            print(tab * 5 + f"test  score: {best_avg_scores[score][0]} with variance: {best_avg_scores[score][1]}")

            rf = RandomForestClassifier(max_depth = best_avg_scores[score][4], min_samples_leaf = best_avg_scores[score][5], random_state = randomstate)
            
            rf.fit(X_train, y_train)            
            y_train_pred, y_test_pred = rf.predict(X_train), rf.predict(X_test)                          
            rmse_train, rmse_test = math.sqrt(mean_squared_error(y_train, y_train_pred)), math.sqrt(mean_squared_error(y_test, y_test_pred))                    
            log_loss_train, log_loss_test = log_loss(y_train, y_train_pred), log_loss(y_test, y_test_pred)        

            score_train, score_test = get_scorer(score)(rf, X_train, y_train), get_scorer(score)(rf, X_test, y_test)
            
            print(tab * 4 + f"Refitted train score: {score_train},  RMSE: {rmse_train}, Log-Loss:{log_loss_train}")
            print(tab * 4 + f"Refitted test  score: {score_test},  RMSE: {rmse_test}, Log-Loss:{log_loss_test}")
            
            n = len(results)
            results.at[n, 'score'] = score
            results.at[n, 'test score'] = best_avg_scores[score][0]
            results.at[n, 'train score'] = best_avg_scores[score][2]
            results.at[n, 'test variance'] = best_avg_scores[score][1]
            results.at[n, 'train variance'] = best_avg_scores[score][3]
            results.at[n, 'test rmse'] = rmse_test
            results.at[n, 'train rmse'] = rmse_train
            results.at[n, 'test log_loss'] = log_loss_test
            results.at[n, 'train log_loss'] = log_loss_train
            results.at[n, 'test size'] = testsize
            results.at[n, 'random state'] = randomstate
            results.at[n, 'estimator'] = "Smote/RandomForestClassifier"
            results.at[n, 'estimator params'] = ','.join([str(p) for p in best_avg_scores[score][4:]])

test size: 0.08
    random state: 250
      f1
        CV score: 0.9728516419814245 using:10,3
          train score: 0.9911814199399192 with variance: 1.0487225559813789e-05
          test  score: 0.9728516419814245 with variance: 0.0005720398485385564
        Refitted train score: 0.9911894273127753,  RMSE: 0.09345386270319955, Log-Loss:0.30164870213895895
        Refitted test  score: 0.9500000000000001,  RMSE: 0.2773500981126146, Log-Loss:2.656859707201984
      accuracy
        CV score: 0.9737219302436694 using:10,3
          train score: 0.9912657643572906 with variance: 1.0146220192779643e-05
          test  score: 0.9737219302436694 with variance: 0.0005118248460445013
        Refitted train score: 0.9912663755458515,  RMSE: 0.09345386270319955, Log-Loss:0.30164870213895895
        Refitted test  score: 0.9230769230769231,  RMSE: 0.2773500981126146, Log-Loss:2.656859707201984
    random state: 650
      f1
        CV score: 0.9596336205497238 using:10,10
          train score:

        Refitted train score: 0.9669603524229075,  RMSE: 0.18176811485266747, Log-Loss:1.1411507610596714
        Refitted test  score: 0.8787878787878788,  RMSE: 0.3481553119113957, Log-Loss:4.186566811348581
    random state: 1050
      f1
        CV score: 0.978017695272214 using:10,3
          train score: 0.9854748167454529 with variance: 4.369961803331331e-06
          test  score: 0.978017695272214 with variance: 0.0005167040110722067
        Refitted train score: 0.9866071428571429,  RMSE: 0.11496024978590211, Log-Loss:0.45645959993274116
        Refitted test  score: 0.9565217391304348,  RMSE: 0.24618298195866548, Log-Loss:2.093307635899448
      accuracy
        CV score: 0.977997557997558 using:10,3
          train score: 0.9856840129567402 with variance: 4.21016132577297e-06
          test  score: 0.977997557997558 with variance: 0.0005308024765900224
        Refitted train score: 0.986784140969163,  RMSE: 0.11496024978590211, Log-Loss:0.45645959993274116
        Refitted t

        Refitted train score: 0.9929742388758782,  RMSE: 0.08372183582789214, Log-Loss:0.24209609528542675
        Refitted test  score: 0.9393939393939393,  RMSE: 0.2857142857142857, Log-Loss:2.819491950604955
      accuracy
        CV score: 0.9697127222982216 using:10,3
          train score: 0.9924061855318568 with variance: 8.89174104660931e-06
          test  score: 0.9697127222982216 with variance: 0.0005184480154801715
        Refitted train score: 0.9929906542056075,  RMSE: 0.08372183582789214, Log-Loss:0.24209609528542675
        Refitted test  score: 0.9183673469387755,  RMSE: 0.2857142857142857, Log-Loss:2.819491950604955
    random state: 2050
      f1
        CV score: 0.9778771039592584 using:10,3
          train score: 0.9920450969929601 with variance: 2.3002379381041806e-06
          test  score: 0.9778771039592584 with variance: 0.00020361089498059979
        Refitted train score: 0.9926650366748166,  RMSE: 0.08533201859828615, Log-Loss:0.251495944623137
        Refit

In [11]:
results.tail()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params
51,accuracy,0.961766,0.978954,0.000578544,2.68186e-05,0.248069,0.133631,2.12548,0.616766,0.2,1250,Smote/RandomForestClassifier,"10,3"
52,f1,0.972515,0.993238,0.000281813,9.21516e-06,0.248069,0.0855399,2.12548,0.252723,0.2,1850,Smote/RandomForestClassifier,"10,3"
53,accuracy,0.973171,0.993293,0.000261749,8.92326e-06,0.248069,0.0855399,2.12548,0.252723,0.2,1850,Smote/RandomForestClassifier,"10,3"
54,f1,0.980721,0.990785,0.00068764,1.05954e-05,0,0.0883883,9.99201e-16,0.269834,0.2,2050,Smote/RandomForestClassifier,"10,3"
55,accuracy,0.981818,0.990886,0.000580199,1.01777e-05,0,0.0883883,9.99201e-16,0.269834,0.2,2050,Smote/RandomForestClassifier,"10,3"


In [12]:
#kidronclassifier
max_estimators = [8, 10, 12]
base_estimators = [LogisticRegression(), KNeighborsClassifier(), DecisionTreeClassifier(), SVC()]
min_rows = [28, 30, 32]
min_columns = [None] #500, 600, 700]
    
for testsize in testsizes:
  print(f"test size: {testsize}")
    
  for randomstate in randomstates:
        print(tab * 2 + f"random state: {randomstate}")
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state = randomstate)
        X_train, X_test = dominant_features(X_train, X_test, y_train, testsize, features)
        
        #no need for treating imbalance here, the classifier does it on its own
        #smote = SMOTE(random_state = randomstate)
        #X_train, y_train = smote.fit_resample(X_train, y_train)

        robustscaler = RobustScaler(quantile_range = (1, 99))
        robustscaler.fit(X_train)
                                    
        X_train = pd.DataFrame(robustscaler.transform(X_train))
        X_test  = pd.DataFrame(robustscaler.transform(X_test))

        #best_scores = {score : [None] for score in scores}
        best_avg_scores = {score : [None] for score in scores}
        
        # Run Grid
        for max_estimator in max_estimators:
            for base_estimator in base_estimators:
                for min_row in min_rows:
                    for min_column in min_columns:
       
                        kc = KidronClassifier(max_estimators = max_estimator, base_estimator = base_estimator, min_rows = min_row, min_columns = min_column, random_state = randomstate)                           
                        cv_results = cross_validate(kc, X_train, y_train, cv = cv, scoring = scores, return_train_score = True, n_jobs = jobs)
 
                        for score in scores:
                            avg_score_test = np.mean(cv_results['test_' + score])
                            var_score_test = np.var(cv_results['test_' + score])
                            avg_score_train = np.mean(cv_results['train_' + score])
                            var_score_train = np.var(cv_results['train_' + score])
                        
                            if(best_avg_scores[score][0] is None or avg_score_test > best_avg_scores[score][0]):
                                best_avg_scores[score] = [avg_score_test, var_score_test, avg_score_train, var_score_train, max_estimator, base_estimator, min_row, min_column]

                    
            
        for score in scores: 
            
            print(tab * 3 + str(score))
            print(tab * 4 + f"CV score: {best_avg_scores[score][0]} using:" + ','.join([str(p) for p in best_avg_scores[score][4:]]))
            print(tab * 5 + f"train score: {best_avg_scores[score][2]} with variance: {best_avg_scores[score][3]}")
            print(tab * 5 + f"test  score: {best_avg_scores[score][0]} with variance: {best_avg_scores[score][1]}")

            kc = KidronClassifier(max_estimators = best_avg_scores[score][4], base_estimator = best_avg_scores[score][5], min_rows = best_avg_scores[score][6], min_columns = best_avg_scores[score][7], random_state = randomstate)
            
            kc.fit(X_train, y_train)            
            y_train_pred, y_test_pred = kc.predict(X_train), kc.predict(X_test)                          
            rmse_train, rmse_test = math.sqrt(mean_squared_error(y_train, y_train_pred)), math.sqrt(mean_squared_error(y_test, y_test_pred))                    
            log_loss_train, log_loss_test = log_loss(y_train, y_train_pred), log_loss(y_test, y_test_pred)        

            score_train, score_test = get_scorer(score)(kc, X_train, y_train), get_scorer(score)(kc, X_test, y_test)
            
            print(tab * 4 + f"Refitted train score: {score_train},  RMSE: {rmse_train}, Log-Loss:{log_loss_train}")
            print(tab * 4 + f"Refitted test  score: {score_test},  RMSE: {rmse_test}, Log-Loss:{log_loss_test}")

            n = len(results)
            results.at[n, 'score'] = score
            results.at[n, 'test score'] = best_avg_scores[score][0]
            results.at[n, 'train score'] = best_avg_scores[score][2]
            results.at[n, 'test variance'] = best_avg_scores[score][1]
            results.at[n, 'train variance'] = best_avg_scores[score][3]
            results.at[n, 'test rmse'] = rmse_test
            results.at[n, 'train rmse'] = rmse_train
            results.at[n, 'test log_loss'] = log_loss_test
            results.at[n, 'train log_loss'] = log_loss_train
            results.at[n, 'test size'] = testsize
            results.at[n, 'random state'] = randomstate
            results.at[n, 'estimator'] = "KidronClassifier"
            results.at[n, 'estimator params'] = ','.join([str(p) for p in best_avg_scores[score][4:]])

test size: 0.08
    random state: 250
      f1
        CV score: 0.9682853053469485 using:12,DecisionTreeClassifier(),32,None
          train score: 0.9725439712940638 with variance: 9.551950487955732e-05
          test  score: 0.9682853053469485 with variance: 0.0009517688028671496
        Refitted train score: 0.9661399548532731,  RMSE: 0.2243559013482789, Log-Loss:1.738529013166646
        Refitted test  score: 0.9743589743589743,  RMSE: 0.19611613513818404, Log-Loss:1.328414476727335
      accuracy
        CV score: 0.9528248587570621 using:12,DecisionTreeClassifier(),32,None
          train score: 0.9589008825287436 with variance: 0.00021949045534632735
          test  score: 0.9528248587570621 with variance: 0.0021201761945800995
        Refitted train score: 0.9496644295302014,  RMSE: 0.2243559013482789, Log-Loss:1.738529013166646
        Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.328414476727335
    random state: 650
      f1
        CV sco

      f1
        CV score: 0.9149190471099671 using:8,LogisticRegression(),30,None
          train score: 0.9211640690227968 with variance: 3.4208106377020326e-05
          test  score: 0.9149190471099671 with variance: 0.0010914707354357804
        Refitted train score: 0.9223529411764706,  RMSE: 0.3367522262074448, Log-Loss:3.9167739526698053
        Refitted test  score: 0.9545454545454546,  RMSE: 0.24618298195866548, Log-Loss:2.0932834056742906
      accuracy
        CV score: 0.8761542957334892 using:8,LogisticRegression(),28,None
          train score: 0.8848712446351932 with variance: 4.92272836117817e-05
          test  score: 0.8761542957334892 with variance: 0.0023292881588210627
        Refitted train score: 0.8831615120274914,  RMSE: 0.3418164536304661, Log-Loss:4.035466653674413
        Refitted test  score: 0.9393939393939394,  RMSE: 0.24618298195866548, Log-Loss:2.0932834056742906
    random state: 1050
      f1
        CV score: 0.9718501633065951 using:8,LogisticRegres

      f1
        CV score: 0.8462970409819726 using:12,LogisticRegression(),28,None
          train score: 0.8428899916647653 with variance: 0.00020322697449186338
          test  score: 0.8462970409819726 with variance: 0.0015919991043137223
        Refitted train score: 0.8435013262599469,  RMSE: 0.4631905164675271, Log-Loss:7.41016074210608
        Refitted test  score: 0.8857142857142858,  RMSE: 0.40406101782088427, Log-Loss:5.638983901209909
      accuracy
        CV score: 0.7890909090909092 using:10,LogisticRegression(),28,None
          train score: 0.7809090909090909 with variance: 0.00016033057851239695
          test  score: 0.7890909090909092 with variance: 0.0031206611570247897
        Refitted train score: 0.7818181818181819,  RMSE: 0.46709936649691375, Log-Loss:7.535756292633027
        Refitted test  score: 0.8367346938775511,  RMSE: 0.40406101782088427, Log-Loss:5.638983901209909
    random state: 1850
      f1
        CV score: 0.9803644157666209 using:10,DecisionTree

        Refitted train score: 0.9652509652509652,  RMSE: 0.18641092980036, Log-Loss:1.2001922283846587
        Refitted test  score: 0.9230769230769231,  RMSE: 0.2773500981126146, Log-Loss:2.6568412549535947
    random state: 2050
      f1
        CV score: 0.9584525059405749 using:8,DecisionTreeClassifier(),30,None
          train score: 0.9683783620332089 with variance: 5.150514524198165e-05
          test  score: 0.9584525059405749 with variance: 0.0009321218954365454
        Refitted train score: 0.967741935483871,  RMSE: 0.21524880100025257, Log-Loss:1.6002521881811909
        Refitted test  score: 0.9911504424778761,  RMSE: 0.12403473458920845, Log-Loss:0.5313657906909345
      accuracy
        CV score: 0.9423076923076923 using:8,DecisionTreeClassifier(),30,None
          train score: 0.9546311780007433 with variance: 9.912069437177304e-05
          test  score: 0.9423076923076923 with variance: 0.0016272189349112412
        Refitted train score: 0.9536679536679536,  RMSE: 0.215

In [13]:
results.tail()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params
107,accuracy,0.934465,0.938234,0.000820372,8.62884e-05,0.248069,0.263625,2.12548,2.40038,0.2,1250,KidronClassifier,"12,KNeighborsClassifier(),30,None"
108,f1,0.979731,0.974949,0.000103124,2.97957e-05,0.27735,0.186411,2.65684,1.20019,0.2,1850,KidronClassifier,"8,KNeighborsClassifier(),28,None"
109,accuracy,0.969231,0.961399,0.000236686,6.44568e-05,0.27735,0.186411,2.65684,1.20019,0.2,1850,KidronClassifier,"8,KNeighborsClassifier(),28,None"
110,f1,0.958453,0.968378,0.000932122,5.15051e-05,0.124035,0.215249,0.531366,1.60025,0.2,2050,KidronClassifier,"8,DecisionTreeClassifier(),30,None"
111,accuracy,0.942308,0.954631,0.00162722,9.91207e-05,0.124035,0.215249,0.531366,1.60025,0.2,2050,KidronClassifier,"8,DecisionTreeClassifier(),30,None"


In [14]:
#logistic parameters
C = [1.0, 0.001, 1.5]

for testsize in testsizes:
  print(f"test size: {testsize}")
    
  for randomstate in randomstates:
        print(tab * 2 + f"random state: {randomstate}")
            
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state = randomstate)
        X_train, X_test = dominant_features(X_train, X_test, y_train, testsize, features)
        
        smote = SMOTE(random_state = randomstate)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        robustscaler = RobustScaler(quantile_range = (1, 99))
        robustscaler.fit(X_train)
                                    
        X_train = robustscaler.transform(X_train)
        X_test  = robustscaler.transform(X_test)

        best_avg_scores = {score : [None] for score in scores}
        
        # Run Grid search for each classifier
        for c in C:            
            lr = LogisticRegression(C = c, solver = 'liblinear', max_iter = 200, class_weight = 'balanced', random_state = randomstate)

            for b in bootstrap:
                for bf in bootstrap_features:
                    for n_estimator in n_estimators:
                        for max_sample in max_samples:

                            bc = BaggingClassifier(base_estimator = lr, bootstrap = b, bootstrap_features = bf, n_estimators = n_estimator, max_samples = max_sample, n_jobs = jobs, random_state = randomstate)
                            cv_results = cross_validate(bc, X_train, y_train, cv = cv, scoring = scores, return_train_score = True, n_jobs = jobs)
                                
                            for score in scores:
                                avg_score_test = np.mean(cv_results['test_' + score])
                                var_score_test = np.var(cv_results['test_' + score])
                                avg_score_train = np.mean(cv_results['train_' + score])
                                var_score_train = np.var(cv_results['train_' + score])

                                if(best_avg_scores[score][0] is None or avg_score_test > best_avg_scores[score][0]):
                                    best_avg_scores[score] = [avg_score_test, var_score_test, avg_score_train, var_score_train, c, b, bf, n_estimator, max_sample]

            
            for score in scores: 
            
                print(tab * 3 + str(score))
                print(tab * 4 + f"CV score: {best_avg_scores[score][0]} using:" + ','.join([str(p) for p in best_avg_scores[score][4:]]))
                print(tab * 5 + f"train score: {best_avg_scores[score][2]} with variance: {best_avg_scores[score][3]}")
                print(tab * 5 + f"test  score: {best_avg_scores[score][0]} with variance: {best_avg_scores[score][1]}")

                random.seed(randomstate)
                lr = LogisticRegression(C = best_avg_scores[score][4], solver = 'liblinear', max_iter = 200, class_weight = 'balanced', random_state = randomstate)
                bc = BaggingClassifier(base_estimator = lr, bootstrap = best_avg_scores[score][5], bootstrap_features = best_avg_scores[score][6], n_estimators = best_avg_scores[score][7], max_samples = best_avg_scores[score][8], n_jobs = jobs, random_state = randomstate)
                    
                bc.fit(X_train, y_train)            
                y_train_pred, y_test_pred = bc.predict(X_train), bc.predict(X_test)                          
                rmse_train, rmse_test = math.sqrt(mean_squared_error(y_train, y_train_pred)), math.sqrt(mean_squared_error(y_test, y_test_pred))                    
                log_loss_train, log_loss_test = log_loss(y_train, y_train_pred), log_loss(y_test, y_test_pred)        

                score_train, score_test = get_scorer(score)(bc, X_train, y_train), get_scorer(score)(bc, X_test, y_test)

                print(tab * 4 + f"Refitted train score: {score_train},  RMSE: {rmse_train}, Log-Loss:{log_loss_train}")
                print(tab * 4 + f"Refitted test  score: {score_test},  RMSE: {rmse_test}, Log-Loss:{log_loss_test}") 

                n = len(results)
                results.at[n, 'score'] = score
                results.at[n, 'test score'] = best_avg_scores[score][0]
                results.at[n, 'train score'] = best_avg_scores[score][2]
                results.at[n, 'test variance'] = best_avg_scores[score][1]
                results.at[n, 'train variance'] = best_avg_scores[score][3]
                results.at[n, 'test rmse'] = rmse_test
                results.at[n, 'train rmse'] = rmse_train
                results.at[n, 'test log_loss'] = log_loss_test
                results.at[n, 'train log_loss'] = log_loss_train
                results.at[n, 'test size'] = testsize
                results.at[n, 'random state'] = randomstate
                results.at[n, 'estimator'] = "Smote/Bagging/LogisticRegression"
                results.at[n, 'estimator params'] = ','.join([str(p) for p in best_avg_scores[score][4:]])

test size: 0.08
    random state: 250
      f1
        CV score: 0.9677928112005191 using:1.0,True,False,2,1.0
          train score: 0.9713396591586289 with variance: 2.4543861692548494e-05
          test  score: 0.9677928112005191 with variance: 0.0007039792792415955
        Refitted train score: 0.9684684684684683,  RMSE: 0.17483616785299216, Log-Loss:1.0557704574863536
        Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      accuracy
        CV score: 0.969326325848065 using:1.0,True,True,30,0.8
          train score: 0.9694257083724184 with variance: 3.728535958532021e-05
          test  score: 0.969326325848065 with variance: 0.0006489586360377187
        Refitted train score: 0.9694323144104804,  RMSE: 0.17483616785299216, Log-Loss:1.0557704574863536
        Refitted test  score: 0.9615384615384616,  RMSE: 0.19611613513818404, Log-Loss:1.328414476727335
      f1
        CV score: 0.9677928112005191 using:1.0,True,False,2,1.0
          train score: 0.97

      f1
        CV score: 0.9388697685656442 using:0.001,False,True,10,0.6
          train score: 0.940540257840353 with variance: 9.199869330900565e-05
          test  score: 0.9388697685656442 with variance: 0.00026359881167508216
        Refitted train score: 0.9388646288209607,  RMSE: 0.24618298195866548, Log-Loss:2.0932799442135535
        Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      accuracy
        CV score: 0.9415147265077138 using:0.001,False,True,10,0.6
          train score: 0.9426338533655606 with variance: 8.387903314025152e-05
          test  score: 0.9415147265077138 with variance: 0.00021929936719194162
        Refitted train score: 0.9393939393939394,  RMSE: 0.24618298195866548, Log-Loss:2.0932799442135535
        Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      f1
        CV score: 0.9421217962447347 using:1.5,True,True,20,1.0
          train score: 0.9409281275347559 with variance: 4.0155598363415644e-05
    

        Refitted train score: 0.9647577092511013,  RMSE: 0.18772930178557284, Log-Loss:1.2172291222762812
        Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      f1
        CV score: 0.9660624822120785 using:0.001,False,False,30,0.6
          train score: 0.9633840559629763 with variance: 2.846207346538037e-06
          test  score: 0.9660624822120785 with variance: 5.738968615787373e-05
        Refitted train score: 0.9638009049773755,  RMSE: 0.18772930178557284, Log-Loss:1.2172291222762812
        Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626413e-16
      accuracy
        CV score: 0.966984126984127 using:0.001,False,False,30,0.6
          train score: 0.9642085187539733 with variance: 2.9206571424683614e-06
          test  score: 0.966984126984127 with variance: 4.723947874130991e-05
        Refitted train score: 0.9647577092511013,  RMSE: 0.18772930178557284, Log-Loss:1.2172291222762812
        Refitted test  score: 1.0,  RMSE: 0.0, Log-

      f1
        CV score: 0.9099923698881327 using:0.001,True,False,20,0.8
          train score: 0.9011377737141224 with variance: 0.0003983942598877862
          test  score: 0.9099923698881327 with variance: 0.0014303880427568947
        Refitted train score: 0.883054892601432,  RMSE: 0.32852627812475244, Log-Loss:3.727765728045452
        Refitted test  score: 0.9767441860465117,  RMSE: 0.17407765595569785, Log-Loss:1.046629587724567
      accuracy
        CV score: 0.9141880341880342 using:0.001,True,False,20,0.8
          train score: 0.9058502103956648 with variance: 0.0003046726365556297
          test  score: 0.9141880341880342 with variance: 0.0011221785404935609
        Refitted train score: 0.8920704845814978,  RMSE: 0.32852627812475244, Log-Loss:3.727765728045452
        Refitted test  score: 0.9696969696969697,  RMSE: 0.17407765595569785, Log-Loss:1.046629587724567
    random state: 1050
      f1
        CV score: 0.9600921550359752 using:1.0,True,True,2,0.8
          tr

        Refitted train score: 0.952914798206278,  RMSE: 0.21699124819614724, Log-Loss:1.6262760131787126
        Refitted test  score: 0.9393939393939394,  RMSE: 0.24618298195866548, Log-Loss:2.093259175449133
    random state: 2050
      f1
        CV score: 0.9717511278455868 using:1.0,True,False,2,1.0
          train score: 0.9720520082584312 with variance: 9.226861516265116e-06
          test  score: 0.9717511278455868 with variance: 0.00027754503403811334
        Refitted train score: 0.9720930232558139,  RMSE: 0.1651445647689541, Log-Loss:0.9419684462189973
        Refitted test  score: 0.9824561403508771,  RMSE: 0.17407765595569785, Log-Loss:1.046629587724567
      accuracy
        CV score: 0.9727272727272729 using:1.0,True,False,2,1.0
          train score: 0.9727272727272729 with variance: 8.393595041322413e-06
          test  score: 0.9727272727272729 with variance: 0.00023760330578512356
        Refitted train score: 0.9727272727272728,  RMSE: 0.1651445647689541, Log-Loss:0

      f1
        CV score: 0.951151410466778 using:1.0,False,True,2,1.0
          train score: 0.952030421375348 with variance: 1.664833130927765e-05
          test  score: 0.951151410466778 with variance: 0.00036735680761559874
        Refitted train score: 0.9539951573849877,  RMSE: 0.21118934387685415, Log-Loss:1.540467488956793
        Refitted test  score: 0.927536231884058,  RMSE: 0.31943828249996997, Log-Loss:3.524381256571095
      accuracy
        CV score: 0.95296853625171 using:1.0,False,True,2,1.0
          train score: 0.9536346386061757 with variance: 1.5540257464252226e-05
          test  score: 0.95296853625171 with variance: 0.00033411420369375744
        Refitted train score: 0.9553990610328639,  RMSE: 0.21118934387685415, Log-Loss:1.540467488956793
        Refitted test  score: 0.8979591836734694,  RMSE: 0.31943828249996997, Log-Loss:3.524381256571095
      f1
        CV score: 0.964300896489096 using:0.001,False,False,20,0.6
          train score: 0.9619201904941459

        Refitted train score: 0.9689737470167065,  RMSE: 0.17428089905580857, Log-Loss:1.0490787203941578
        Refitted test  score: 0.9393939393939393,  RMSE: 0.2857142857142857, Log-Loss:2.819491950604955
      accuracy
        CV score: 0.967359781121751 using:1.0,True,True,10,0.8
          train score: 0.9725504236782433 with variance: 8.658252884526905e-06
          test  score: 0.967359781121751 with variance: 0.0003980649785444674
        Refitted train score: 0.9719626168224299,  RMSE: 0.16744367165578428, Log-Loss:0.9683806447051144
        Refitted test  score: 0.9387755102040817,  RMSE: 0.24743582965269675, Log-Loss:2.1146189629537164
      f1
        CV score: 0.9669995204582253 using:1.0,True,True,2,0.8
          train score: 0.9685070197971566 with variance: 1.9822288696846764e-05
          test  score: 0.9669995204582253 with variance: 0.0002914653380400275
        Refitted train score: 0.9689737470167065,  RMSE: 0.17428089905580857, Log-Loss:1.0490787203941578
      

      f1
        CV score: 0.9562289562289562 using:1.0,True,True,2,1.0
          train score: 0.9560703812316715 with variance: 6.824846707544894e-06
          test  score: 0.9562289562289562 with variance: 0.00011253421546831739
        Refitted train score: 0.9560723514211887,  RMSE: 0.20513217183267982, Log-Loss:1.4533643532511933
        Refitted test  score: 0.9318181818181819,  RMSE: 0.3038218101251, Log-Loss:3.1881947441456027
      accuracy
        CV score: 0.9579320987654321 using:1.0,True,True,2,1.0
          train score: 0.9579214921836181 with variance: 6.071036382041621e-06
          test  score: 0.9579320987654321 with variance: 9.666590458771543e-05
        Refitted train score: 0.9579207920792079,  RMSE: 0.20513217183267982, Log-Loss:1.4533643532511933
        Refitted test  score: 0.9076923076923077,  RMSE: 0.3038218101251, Log-Loss:3.1881947441456027
      f1
        CV score: 0.9562289562289562 using:1.0,True,True,2,1.0
          train score: 0.9560703812316715 wit

      f1
        CV score: 0.9407017876638128 using:1.0,False,False,10,0.6
          train score: 0.9386604072209541 with variance: 5.510897069258975e-05
          test  score: 0.9407017876638128 with variance: 0.001495560540075646
        Refitted train score: 0.9435897435897437,  RMSE: 0.23690177073967142, Log-Loss:1.9384211139345338
        Refitted test  score: 0.9615384615384616,  RMSE: 0.2480694691784169, Log-Loss:2.1254754642626614
      accuracy
        CV score: 0.9414151249594289 using:0.001,False,True,20,0.6
          train score: 0.9317637003723979 with variance: 4.2698212087555034e-05
          test  score: 0.9414151249594289 with variance: 0.0013799015882015398
        Refitted train score: 0.9413265306122449,  RMSE: 0.24222607082590245, Log-Loss:2.0265302373909386
        Refitted test  score: 0.9230769230769231,  RMSE: 0.2773500981126146, Log-Loss:2.6568535564525213
    random state: 1850
      f1
        CV score: 0.9608424986695091 using:1.0,False,False,2,1.0
        

In [15]:
results.tail()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params
275,accuracy,0.961005,0.957039,0.000470484,5.17416e-05,0.124035,0.197642,0.531366,1.34917,0.2,2050,Smote/Bagging/LogisticRegression,"1.0,True,True,20,0.8"
276,f1,0.959435,0.95705,0.000328069,5.5026e-05,0.124035,0.197642,0.531366,1.34918,0.2,2050,Smote/Bagging/LogisticRegression,"1.0,False,True,2,0.8"
277,accuracy,0.961005,0.957039,0.000470484,5.17416e-05,0.124035,0.197642,0.531366,1.34917,0.2,2050,Smote/Bagging/LogisticRegression,"1.0,True,True,20,0.8"
278,f1,0.961901,0.958744,0.000365189,3.79716e-05,0.124035,0.204124,0.531366,1.43912,0.2,2050,Smote/Bagging/LogisticRegression,"1.5,True,True,2,0.8"
279,accuracy,0.963602,0.96029,0.000295253,3.10624e-05,0.124035,0.204124,0.531366,1.43912,0.2,2050,Smote/Bagging/LogisticRegression,"1.5,True,True,2,0.8"


In [16]:
#bagging svc
kernels = ['linear', 'rbf']
C = [0.001, 1, 1.5]

for testsize in testsizes:
  print(f"test size: {testsize}")
    
  for randomstate in randomstates:
        print(tab * 2 + f"random state: {randomstate}")
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testsize, random_state = randomstate)
        X_train, X_test = dominant_features(X_train, X_test, y_train, testsize, features)

        smote = SMOTE(random_state = randomstate)
        X_train, y_train = smote.fit_resample(X_train, y_train)

        robustscaler = RobustScaler(quantile_range = (1, 99))
        robustscaler.fit(X_train)
                                    
        X_train = robustscaler.transform(X_train)
        X_test  = robustscaler.transform(X_test)

        best_avg_scores = {score : [None] for score in scores}
        
        # Run Grid search for each classifier
        for kernel in kernels:
            for c in C:
                    
                svc = SVC(kernel = kernel, C = c, max_iter = 200, class_weight = 'balanced', random_state = randomstate)

                for b in bootstrap:
                    for bf in bootstrap_features:
                        for n_estimator in n_estimators:
                            for max_sample in max_samples:

                                bc = BaggingClassifier(base_estimator = svc, bootstrap = b, bootstrap_features = bf, n_estimators = n_estimator, max_samples = max_sample, n_jobs = jobs, random_state = randomstate)
                                cv_results = cross_validate(bc, X_train, y_train, cv = cv, scoring = scores, return_train_score = True, n_jobs = jobs)

                                for score in scores:
                                    avg_score_test = np.mean(cv_results['test_' + score])
                                    var_score_test = np.var(cv_results['test_' + score])
                                    avg_score_train = np.mean(cv_results['train_' + score])
                                    var_score_train = np.var(cv_results['train_' + score])

                                    if(best_avg_scores[score][0] is None or avg_score_test > best_avg_scores[score][0]):
                                        best_avg_scores[score] = [avg_score_test, var_score_test, avg_score_train, var_score_train, b, bf, n_estimator, max_sample, kernel, c]


        for score in scores: 
            
            print(tab * 3 + str(score))
            print(tab * 4 + f"CV score: {best_avg_scores[score][0]} using:" + ','.join([str(p) for p in best_avg_scores[score][4:]]))
            print(tab * 5 + f"train score: {best_avg_scores[score][2]} with variance: {best_avg_scores[score][3]}")
            print(tab * 5 + f"test  score: {best_avg_scores[score][0]} with variance: {best_avg_scores[score][1]}")

            svc = SVC(kernel = best_avg_scores[score][8], C = best_avg_scores[score][9], max_iter = 200, class_weight = 'balanced', random_state = randomstate)
            bc = BaggingClassifier(base_estimator = svc, bootstrap = best_avg_scores[score][4], bootstrap_features = best_avg_scores[score][5], n_estimators = best_avg_scores[score][6], max_samples = best_avg_scores[score][7], n_jobs = jobs)
            
            bc.fit(X_train, y_train)            
            y_train_pred, y_test_pred = bc.predict(X_train), bc.predict(X_test)                          
            rmse_train, rmse_test = math.sqrt(mean_squared_error(y_train, y_train_pred)), math.sqrt(mean_squared_error(y_test, y_test_pred))                    
            log_loss_train, log_loss_test = log_loss(y_train, y_train_pred), log_loss(y_test, y_test_pred)        

            score_train, score_test = get_scorer(score)(bc, X_train, y_train), get_scorer(score)(bc, X_test, y_test)
            
            print(tab * 4 + f"Refitted train score: {score_train},  RMSE: {rmse_train}, Log-Loss:{log_loss_train}")
            print(tab * 4 + f"Refitted test  score: {score_test},  RMSE: {rmse_test}, Log-Loss:{log_loss_test}")            
            
            n = len(results)
            results.at[n, 'score'] = score
            results.at[n, 'test score'] = best_avg_scores[score][0]
            results.at[n, 'train score'] = best_avg_scores[score][2]
            results.at[n, 'test variance'] = best_avg_scores[score][1]
            results.at[n, 'train variance'] = best_avg_scores[score][3]
            results.at[n, 'test rmse'] = rmse_test
            results.at[n, 'train rmse'] = rmse_train
            results.at[n, 'test log_loss'] = log_loss_test
            results.at[n, 'train log_loss'] = log_loss_train
            results.at[n, 'test size'] = testsize
            results.at[n, 'random state'] = randomstate
            results.at[n, 'estimator'] = "Smote/Bagging/SVC"
            results.at[n, 'estimator params'] = ','.join([str(p) for p in best_avg_scores[score][4:]])

test size: 0.08
    random state: 250
      f1
        CV score: 0.9536160491182979 using:True,False,2,1.0,rbf,1
          train score: 0.9484053573125888 with variance: 4.3703583605295293e-05
          test  score: 0.9536160491182979 with variance: 0.0010782845071462192
        Refitted train score: 0.9523809523809524,  RMSE: 0.21412969991171116, Log-Loss:1.583659177921365
        Refitted test  score: 0.9743589743589743,  RMSE: 0.19611613513818404, Log-Loss:1.328414476727335
      accuracy
        CV score: 0.9562111801242235 using:True,False,2,1.0,rbf,1
          train score: 0.9503208707434375 with variance: 3.7603290686529364e-05
          test  score: 0.9562111801242235 with variance: 0.0009172933051700794
        Refitted train score: 0.9606986899563319,  RMSE: 0.19824558013652693, Log-Loss:1.3574226513171463
        Refitted test  score: 0.9230769230769231,  RMSE: 0.2773500981126146, Log-Loss:2.656859707201984
    random state: 650
      f1
        CV score: 0.9666357322754567 

        CV score: 0.9208058608058607 using:False,True,20,0.8,rbf,1.5
          train score: 0.9190582145127599 with variance: 4.941045016671892e-05
          test  score: 0.9208058608058607 with variance: 0.0016065907230742402
        Refitted train score: 0.920704845814978,  RMSE: 0.28159395267835924, Log-Loss:2.738766405735541
        Refitted test  score: 0.9090909090909091,  RMSE: 0.30151134457776363, Log-Loss:3.139937223624014
    random state: 1050
      f1
        CV score: 0.9708325382482685 using:False,True,30,0.8,rbf,1.5
          train score: 0.9696213023996574 with variance: 1.4340865371218624e-05
          test  score: 0.9708325382482685 with variance: 0.00022971856506646444
        Refitted train score: 0.9661399548532731,  RMSE: 0.18176811485266747, Log-Loss:1.141152522287491
        Refitted test  score: 1.0,  RMSE: 0.0, Log-Loss:9.992007221626415e-16
      accuracy
        CV score: 0.9713797313797313 using:False,True,30,0.8,rbf,1.5
          train score: 0.97026458390

        Refitted train score: 0.8523809523809524,  RMSE: 0.3842122429322726, Log-Loss:5.0986136420018475
        Refitted test  score: 0.8163265306122449,  RMSE: 0.4285714285714286, Log-Loss:6.343873207176049
    random state: 1850
      f1
        CV score: 0.980952380952381 using:True,False,20,0.8,linear,1.5
          train score: 0.9791257995735607 with variance: 1.0820781865876242e-05
          test  score: 0.980952380952381 with variance: 9.070294784580434e-05
        Refitted train score: 0.9809523809523809,  RMSE: 0.13671718540493263, Log-Loss:0.6455846055123502
        Refitted test  score: 0.955223880597015,  RMSE: 0.24743582965269675, Log-Loss:2.1146189629537164
      accuracy
        CV score: 0.9813406292749658 using:True,False,20,0.8,linear,1.5
          train score: 0.9795594428247488 with variance: 1.008124469191688e-05
          test  score: 0.9813406292749658 with variance: 8.705425732791157e-05
        Refitted train score: 0.9789719626168224,  RMSE: 0.145010473356849

        Refitted train score: 0.9557291666666666,  RMSE: 0.2104063528825433, Log-Loss:1.5290604133163592
        Refitted test  score: 0.9692307692307692,  RMSE: 0.17541160386140583, Log-Loss:1.0627315813818679


In [17]:
results.tail()

,score,test score,train score,test variance,train variance,test rmse,train rmse,test log_loss,train log_loss,test size,random state,estimator,estimator params
331,accuracy,0.954138,0.954085,0.00113026,9.19026e-05,0.214834,0.225877,1.5941,1.76219,0.2,1250,Smote/Bagging/SVC,"True,True,20,1.0,rbf,1.5"
332,f1,0.982653,0.988273,9.85351e-05,5.46239e-06,0.214834,0.098773,1.59411,0.336964,0.2,1850,Smote/Bagging/SVC,"False,False,2,1.0,rbf,1.5"
333,accuracy,0.982927,0.987195,3.5693e-05,1.26413e-05,0.27735,0.110432,2.65684,0.421205,0.2,1850,Smote/Bagging/SVC,"False,True,20,0.8,rbf,1.5"
334,f1,0.956388,0.960992,0.000886954,9.50902e-05,0.124035,0.204124,0.531366,1.43912,0.2,2050,Smote/Bagging/SVC,"True,False,10,1.0,rbf,1.5"
335,accuracy,0.958442,0.95313,0.000634171,4.01393e-05,0.175412,0.210406,1.06273,1.52906,0.2,2050,Smote/Bagging/SVC,"True,False,2,0.8,linear,1.5"


In [18]:
results.to_csv(path + "classification-results-3-shrink-50.csv")